<a href="https://colab.research.google.com/github/GuanxiongLiu/US-Investment-Tools/blob/main/yfinance_CallPutFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install yfinance

In [3]:
import yfinance as yf

# Preparation

In [4]:
import pandas as pd

basic_stocks = pd.read_csv(
    '<path to demo code or your own code>'
)
basic_stocks.head()

,Ticker,BP_1_date,BP_2_date,Vertical_Dis_Per_Channel
0,SPY,2020-03-23,2021-03-04,4.405
1,QQQ,2020-03-24,2021-05-19,5.457


In [5]:
history_daily = yf.download(# or pdr.get_data_yahoo(...
                            # tickers list or string as well
                            tickers = " ".join(basic_stocks['Ticker'].to_numpy()),

                            # use "period" instead of start/end
                            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                            # (optional, default is '1mo')
                            period = "2y",

                            # fetch data by interval (including intraday if period < 60 days)
                            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                            # (optional, default is '1d')
                            interval = "1d",

                            # group by ticker (to access via data['SPY'])
                            # (optional, default is 'column')
                            group_by = 'ticker',

                            # adjust all OHLC automatically
                            # (optional, default is False)
                            auto_adjust = True,

                            # download pre/post regular market hours data
                            # (optional, default is False)
                            prepost = True,

                            # use threads for mass downloading? (True/False/Integer)
                            # (optional, default is True)
                            threads = True,

                            # proxy URL scheme use use when downloading?
                            # (optional, default is None)
                            proxy = None
                        )

[*********************100%***********************]  2 of 2 completed


In [6]:
dates = [str(d)[0:10] for d in history_daily.index.to_numpy()]
today, dates = dates[-1], dates[:-1]
history_daily = history_daily.drop(index=history_daily.index[-1])

In [7]:
history_daily.tail()

QQQ              ...         SPY          
                  Open        High  ...       Close    Volume
Date                                ...                      
2021-05-20  324.119995  329.709991  ...  415.279999  78022200
2021-05-21  330.429993  330.859985  ...  414.940002  76519100
2021-05-24  329.369995  333.670013  ...  419.170013  51376700
2021-05-25  334.070007  334.809998  ...  418.239990  57451400
2021-05-26  333.660004  334.609985  ...  419.070007  42997900

[5 rows x 10 columns]

# Filtering

In [28]:
call_interest_thres, put_interest_thres = 0.3, 0.7

In [29]:
call_direction, put_direction = [], []
for index in range(basic_stocks.shape[0]):
    row = basic_stocks.iloc[index]
    #
    BP_1_loc, BP_2_loc, TD_loc = dates.index(row['BP_1_date']), dates.index(row['BP_2_date']), history_daily.shape[0]+1
    BP_price_diff = (history_daily.iloc[BP_2_loc][(row['Ticker'], 'Low')] - history_daily.iloc[BP_1_loc][(row['Ticker'], 'Low')])
    RF_price_diff = (TD_loc - BP_1_loc) / (BP_2_loc - BP_1_loc) * BP_price_diff
    RF_price = history_daily.iloc[BP_1_loc][(row['Ticker'], 'Low')] + RF_price_diff
    #
    ticker = yf.Ticker(row['Ticker'])
    price = ticker.history(period='1d', interval='1d').to_numpy()[0,3]
    #
    ratio = (abs(price - RF_price) / row['Vertical_Dis_Per_Channel']) - (abs(price - RF_price) // row['Vertical_Dis_Per_Channel'])
    if ratio <= call_interest_thres:
        call_direction.append(row['Ticker'])
    elif ratio >= put_interest_thres:
        put_direction.append(row['Ticker'])
    else:
        pass

In [30]:
print('The CALL direction Tickers are: \n')
print(call_direction)
print('The PUT direction Tickers are: \n')
print(put_direction)

The CALL direction Tickers are: 

['SPY']
The PUT direction Tickers are: 

[]
